# How to run a model using `fowt_ml` and calculate scores

## Data preparation (common for any type of model)

In [1]:
from fowt_ml.datasets import get_data, fix_column_names
from fowt_ml import Config
from sklearn.model_selection import train_test_split

In [2]:
example_config_file = "../../src/example_config.yml"

In [3]:
config = Config.from_yaml(example_config_file)
config["data"]["exp699"]["path_file"] = "/home/sarah/temp/hybridlabs/data_example/exp699.mat"
df = get_data("exp699", config["data"])

In [4]:
# train/test split
predictors_labels = config["ml_setup"]["predictors"]
target_labels = config["ml_setup"]["targets"]

# rename the column names to exclude []
X_data = df.loc[:, predictors_labels]
Y_data = df.loc[:, target_labels]

X_data, Y_data = fix_column_names(X_data), fix_column_names(Y_data)
train_test_split_kwargs = config["ml_setup"]["train_test_split_kwargs"]
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, **train_test_split_kwargs)

In [5]:
metrics = config["ml_setup"]["metric_names"]
metrics

['neg_mean_absolute_error',
 'neg_root_mean_squared_error',
 'r2',
 'model_fit_time',
 'model_predict_time']

## Linear models 

In [6]:
from fowt_ml import LinearModels
LinearModels.ESTIMATOR_NAMES

{'LinearRegression': sklearn.linear_model._base.LinearRegression,
 'RidgeRegression': sklearn.linear_model._ridge.Ridge,
 'LassoRegression': sklearn.linear_model._coordinate_descent.Lasso,
 'ElasticNetRegression': sklearn.linear_model._coordinate_descent.ElasticNet,
 'LeastAngleRegression': sklearn.linear_model._least_angle.Lars}

In [7]:
# calculate metrics
model_name = "LeastAngleRegression"
model = LinearModels(model_name)

model.use_scaled_data()
scores = model.calculate_score(X_train, X_test, y_train, y_test, metrics) 
scores

{'neg_mean_absolute_error': -1.603052611245188,
 'neg_root_mean_squared_error': -2.1309153094889486,
 'r2': 0.10732388900915783,
 'model_fit_time': np.float64(0.096),
 'model_predict_time': np.float64(0.001)}

In [8]:
scores = model.cross_validate(X_train, y_train, metrics) 
scores

{'model_fit_time': array([0.109, 0.085, 0.081, 0.08 , 0.082]),
 'neg_mean_absolute_error': array([-1.60136019, -1.60960767, -1.6112611 , -1.61425787, -1.59912871]),
 'neg_root_mean_squared_error': array([-2.13342478, -2.13724123, -2.14536043, -2.15210295, -2.1258841 ]),
 'r2': array([0.10649117, 0.10830421, 0.10762699, 0.10839059, 0.10954135]),
 'model_predict_time': array([0.001, 0.001, 0.001, 0.001, 0.001])}

## Random Forest

In [9]:
from fowt_ml import EnsembleModel
EnsembleModel.ESTIMATOR_NAMES

{'ExtraTrees': sklearn.ensemble._forest.ExtraTreesRegressor,
 'RandomForest': sklearn.ensemble._forest.RandomForestRegressor}

In [10]:
model_name = "RandomForest"
model = EnsembleModel(estimator=model_name, max_depth=9, bootstrap=True, max_samples=10_000, n_estimators=50)

model.use_scaled_data()
scores = model.calculate_score(X_train, X_test, y_train, y_test, metrics) 
scores

{'neg_mean_absolute_error': -1.5959389437050893,
 'neg_root_mean_squared_error': -2.1135488771633013,
 'r2': 0.12112219032333066,
 'model_fit_time': np.float64(5.78),
 'model_predict_time': np.float64(0.003)}

## Gaussian Process

In [12]:
from fowt_ml import SparseGaussianModel
SparseGaussianModel.ESTIMATOR_NAMES

{'SklearnGPRegressor': fowt_ml.gaussian_process.SklearnGPRegressor}

In [13]:
model_name = "SklearnGPRegressor"
params = config["ml_setup"]["model_names"][model_name]

model = SparseGaussianModel(model_name, **params)
model.use_scaled_data()
scores = model.calculate_score(X_train, X_test, y_train, y_test, metrics) 
scores

{'neg_mean_absolute_error': -1.5981088588623225,
 'neg_root_mean_squared_error': -2.1218047367573445,
 'r2': 0.12320574147031532,
 'model_fit_time': np.float64(88.066),
 'model_predict_time': np.float64(0.021)}

## MLP

In [14]:
from fowt_ml import NeuralNetwork
NeuralNetwork.ESTIMATOR_NAMES

{'MultilayerPerceptron': sklearn.neural_network._multilayer_perceptron.MLPRegressor}

In [15]:
model_name = "MultilayerPerceptron"
params = config["ml_setup"]["model_names"][model_name]

model = NeuralNetwork(model_name, **params)
model.use_scaled_data()
scores = model.calculate_score(X_train, X_test, y_train, y_test, metrics) 
scores

{'neg_mean_absolute_error': -1.5981261086400942,
 'neg_root_mean_squared_error': -2.1186915415839853,
 'r2': 0.12120288013830806,
 'model_fit_time': np.float64(11.438),
 'model_predict_time': np.float64(0.001)}

## XGBoots

In [16]:
from fowt_ml import XGBoost
XGBoost.ESTIMATOR_NAMES

{'XGBoostRegression': xgboost.sklearn.XGBRegressor}

In [17]:
model_name = "XGBoostRegression"
params = config["ml_setup"]["model_names"][model_name]
model = XGBoost(model_name, **params)

model.use_scaled_data()
scores = model.calculate_score(X_train, X_test, y_train, y_test, metrics) 
scores

{'neg_mean_absolute_error': -1.581722378730774,
 'neg_root_mean_squared_error': -2.0865964889526367,
 'r2': 0.15320807695388794,
 'model_fit_time': np.float64(2.811),
 'model_predict_time': np.float64(0.001)}